In [9]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
from sympy import *
from scipy.io import savemat
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [10]:
# All functions used below
def derivtransfer(FPs,type='tanhp'):
    xe,xi=FPs[0],FPs[1]
    if type=='tanhp':
        derive=1/np.cosh(xe)**2
        derivi=1/np.cosh(xi)**2
        return np.array([derive,derivi])
def tanhpfunc(x):
    return np.tanh(x)+1.0
def checkXbase(FPs,J,type='tanhp'):
    xe,xi=FPs[0],FPs[1]
    if type=='tanhp':
        derxe=-xe+tanhpfunc(xe)*J[0,0]+tanhpfunc(xi)*J[0,1]
        derxi=-xi+tanhpfunc(xe)*J[1,0]+tanhpfunc(xi)*J[1,1]
        return (derxe,derxi)
    elif type=='tanh':
        derxe=-xe+np.tanh(xe)*J[0,0]+np.tanh(xi)*J[0,1]
        derxi=-xi+np.tanh(xe)*J[1,0]+np.tanh(xi)*J[1,1]
        return (derxe,derxi)
def FPfunc(x,*data):
    JE,JI,a,b,tfunc=data
    if tfunc=='tanhp':
        x0 = float(x[0])
        x1 = float(x[1])
        resp0=np.tanh(x0+x1)+1
        resp1=np.tanh(x0-x1)+1
        return [
            x0-JE*resp0+JI*resp1,
            x1+a*resp0-b*resp1
        ]  
    elif tfunc=='tanh':
        x0 = float(x[0])
        x1 = float(x[1])
        resp0=(np.tanh(x0+x1))
        resp1=(np.tanh(x0-x1))
        return [
            x0-JE*resp0+JI*resp1,
            x1+a*resp0-b*resp1
        ]        


def FPfuncSVD(x,*data):
    JE,JI,a,b,tfunc=data
    if tfunc=='tanhp':
        x0 = float(x[0])
        x1 = float(x[1])
        # calculate SVD
        Jt = np.zeros((2,2))
        Jt[:,0],Jt[:,1]=JE,-JI
        Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
        lvec,sv,rvech=la.svd(Jt)
        m,n=lvec,rvech.T
        m[:,0]*=sv[0]
        m[:,1]*=sv[1]
        if n[0,0]<0:
            n[:,0]*=(-1)   
            m[:,0]*=(-1)
        if n[0,1]<0:
            n[:,1]*=(-1)   
            m[:,1]*=(-1)
        resp0=np.tanh(x0*m[0,0]+x1*m[0,1])+1
        resp1=np.tanh(x0*m[1,0]+x1*m[1,1])+1
        return [
            x0-n[0,0]*resp0-n[1,0]*resp1,
            x1-n[0,1]*resp0-n[1,1]*resp1
        ]
    elif tfunc=='tanh':
        x0 = float(x[0])
        x1 = float(x[1])
        # calculate SVD
        Jt = np.zeros((2,2))
        Jt[:,0],Jt[:,1]=JE,-JI
        Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
        lvec,sv,rvech=la.svd(Jt)
        m,n=lvec,rvech.T
        m[:,0]*=sv[0]
        m[:,1]*=sv[1]
        # print('sigma2:',sv[1])
        if n[0,0]<0:
            n[:,0]*=(-1)   
            m[:,0]*=(-1)
        if n[0,1]<0:
            n[:,1]*=(-1)   
            m[:,1]*=(-1)
        resp0=(np.tanh(x0*m[0,0]+x1*m[0,1])+0.0)
        resp1=(np.tanh(x0*m[1,0]+x1*m[1,1])+0.0)
        return [
            x0-n[0,0]*resp0-n[1,0]*resp1,
            x1-n[0,1]*resp0-n[1,1]*resp1
        ]

    
def FPXfunc(x,*data):
    JE,JI,a,b,tfunc=data
    # calculate connectivity matrix
    Jt = np.zeros((2,2))
    Jt[:,0],Jt[:,1]=JE,-JI
    Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
    if tfunc=='tanhp':
        x0 = float(x[0])
        x1 = float(x[1])
        respxE=tanhpfunc(x0)*Jt[0,0]+tanhpfunc(x1)*Jt[0,1]
        respxI=tanhpfunc(x0)*Jt[1,0]+tanhpfunc(x1)*Jt[1,1]
        return [
            x0-respxE,
            x1-respxI
        ]  
    elif tfunc=='tanh':
        x0 = float(x[0])
        x1 = float(x[1])
        respxE=np.tanh(x0)*Jt[0,0]+np.tanh(x1)*Jt[0,1]
        respxI=np.tanh(x0)*Jt[1,0]+np.tanh(x1)*Jt[1,1]
        return [
            x0-respxE,
            x1-respxI
        ]  
        

In [11]:
def dynkappa(M,N,kappaFP,kappaP,timeparams,type='tanhp',express='EI'):
    tfinal,dt=timeparams[0],timeparams[1]
    nt = int(tfinal/dt)
    kappadyn=np.zeros((2,nt))
    kappadyn[:,0]=kappaFP+kappaP
    xactdyn = np.zeros_like(kappadyn)
    xactdyn[:,0]=kappadyn[0,0]*M[:,0]+kappadyn[1,0]*M[:,1]
    eps=1e-5
    if type=='tanhp':
        # check the FP
        xFP=kappaFP[0]*M[:,0]+kappaFP[1]*M[:,1]
        phisyn=np.zeros_like(kappaFP)
        for i in range(2):
            phisyn[i]=tanhpfunc(xFP[i])
        phisyn=np.reshape(phisyn,(2,1))
        devkappa=-np.reshape(kappaFP,(2,1))+np.reshape(N.T@phisyn,(2,1))
        if np.sum(devkappa**2)>eps:
            print(express,' error of kappa FPs:',np.sum(devkappa**2))
            print('FP error')
        for it in range(1,nt):
            xact = kappadyn[0,it-1]*M[:,0]+kappadyn[1,it-1]*M[:,1]
            phisyn=np.reshape(tanhpfunc(xact),(2,1))
            deltakappa=dt*(-kappadyn[:,it-1]+np.squeeze(N.T@phisyn))
            kappadyn[:,it]=kappadyn[:,it-1]+np.squeeze(deltakappa)
            xactdyn[:,it]=kappadyn[0,it]*M[:,0]+kappadyn[1,it]*M[:,1]
    elif type=='tanh':
        # check the FP
        xFP=kappaFP[0]*M[:,0]+kappaFP[1]*M[:,1]
        phisyn=np.zeros_like(kappaFP)
        for i in range(2):
            phisyn[i]=np.tanh(xFP[i])
        phisyn=np.reshape(phisyn,(2,1))
        devkappa=-np.reshape(kappaFP,(2,1))+np.reshape(N.T@phisyn,(2,1))
        if np.sum(devkappa**2)>eps:
            print(express,' error of kappa FPs:',np.sum(devkappa**2))
            print('FP error')
        for it in range(1,nt):
            xact = kappadyn[0,it-1]*M[:,0]+kappadyn[1,it-1]*M[:,1]
            phisyn=np.reshape(np.tanh(xact),(2,1))
            deltakappa=dt*(-kappadyn[:,it-1]+np.squeeze(N.T@phisyn))
            kappadyn[:,it]=kappadyn[:,it-1]+np.squeeze(deltakappa)
            xactdyn[:,it]=kappadyn[0,it]*M[:,0]+kappadyn[1,it]*M[:,1]

    return (kappadyn,xactdyn,devkappa)

def dynxact(J,xFP,xP,timeparams,type='tanhp'):
    tfinal,dt=timeparams[0],timeparams[1]
    nt = int(tfinal/dt)
    xactdyn=np.zeros((2,nt))
    # print(xFP)
    xactdyn[:,0]=xFP+xP
    eps=1e-5
    if type=='tanhp':
        # check the FP
        phisyn=np.zeros_like(xFP)
        for i in range(2):
            phisyn[i]=tanhpfunc(xFP[i])
        phisyn=np.reshape(phisyn,(2,1))
        # print(phisyn)
        devxact=-np.reshape(xFP,(2,1))+np.reshape(J@phisyn,(2,1))
        if np.sum(devxact**2)>eps:
            print('error of FPs:',np.sum(devxact**2))
            print('x activity FP error')
        for it in range(1,nt):
            phisyn=np.reshape(tanhpfunc(xactdyn[:,it-1]),(2,1))
            deltaxact=dt*(-xactdyn[:,it-1]+np.squeeze(J@phisyn))
            xactdyn[:,it]=xactdyn[:,it-1]+deltaxact
    elif type=='tanh':
        # check the FP
        phisyn=np.zeros_like(xFP)
        for i in range(2):
            phisyn[i]=np.tanh(xFP[i])
        phisyn=np.reshape(phisyn,(2,1))
        # print(phisyn)
        devxact=-np.reshape(xFP,(2,1))+np.reshape(J@phisyn,(2,1))
        if np.sum(devxact**2)>eps:
            print('error of FPs:',np.sum(devxact**2))
            print('x activity FP error')
        for it in range(1,nt):
            phisyn=np.reshape(np.tanh(xactdyn[:,it-1]),(2,1))
            deltaxact=dt*(-xactdyn[:,it-1]+np.squeeze(J@phisyn))
            xactdyn[:,it]=xactdyn[:,it-1]+deltaxact
    return (xactdyn,devxact)
# reversely calculate kappaSVD
def RevfuncSVD(x,*data):
    m,xFP,tfunc=data
    if tfunc=='tanhp':
        k0 = float(x[0])
        k1 = float(x[1])
        rate1=tanhpfunc(xFP[0])
        rate2=tanhpfunc(xFP[1])
    
        return [
            k0*m[0,0]+k1*m[0,1]-xFP[0],
            k0*m[1,0]+k1*m[1,1]-xFP[1]
        ]

** Section 3.1 SVD and properties **
* check the identical singular values of connectivity matrix and overlap matrix under two coordinate systems.
* keep either of the parameters a or b and change the other parameter, see the influence on singular values
* check different singular values of transient matrix 
* which one is larger? (determine the amplification)

In [79]:
# 2 by 2 connectivity matrix, rank-1 structure
# transfer function \phi(x)=tanh(x)
'''
J = np.zeros((2,2))
'''
# JE is large enough to result in a bistable regime
JE,JI,a,b=1.3,1.5,0.2,0.60#3.6,1.2,0.5,-0.50#keynote
# EI vectors 
J = np.array([[JE-a,-JI+b],[JE+a,-JI-b]])
M,N=np.array([[1,1],[1,-1]]),np.array([[JE,-a],[-JI,b]])
eigv,eigvec=la.eig(J)
epscheck=1e-6
if (np.imag(eigv[0]) or np.imag(eigv[1]))>epscheck:
    print('eigenvalues:',eigv)
    print('complex eigenvalues!')
Jei=M@N.T
# singular vectors
lvec,sv,rvech=la.svd(J)
m,n=lvec.copy(),rvech.copy()
m[:,0],m[:,1]=m[:,0]*sv[0],m[:,1]*sv[1]
n=n.T
if n[0,0]<0:
    n[:,0]*=(-1)   
    m[:,0]*=(-1)
if n[0,1]<0:
    n[:,1]*=(-1)   
    m[:,1]*=(-1)
Jsvd=m@n.T 
# # Check these 3 matrix
# print('connectivity matrices:',J,Jei,Jsvd)

# define the overlap matrix, under two coordinate systems
Jovl_ei,Jovl_svd=N.T@M,n.T@m
# compare their singular values 
compsigovl=np.zeros((2,3))
_,compsigovl[:,1],_=la.svd(Jovl_ei)
_,compsigovl[:,2],_=la.svd(Jovl_svd)
_,compsigovl[:,0],_=la.svd(J)
# print('comparing singular values of overlap matrices',compsigovl)

# define the transient matrices, under two coordinate systems, and also with original transient matrix
# choose a transfer function, find the fixed points
data=(JE,JI,a,b,'tanhp')

x0=[1.7,0.00]
kappaEI0=[np.sum(x0)/2.0,(x0[0]-x0[1])/2.0]
resultEIk = fsolve(FPfunc,kappaEI0,data)
xFPEI= M@np.reshape(resultEIk,(2,1))

resultx = fsolve(FPXfunc,x0,data)

# reversely find the kappa SVD
xrev0=[0,0]
datarev=(m,resultx,'tanhp')
kappasvdRev=fsolve(RevfuncSVD,xrev0,datarev)
print('reversely kappa_SVD:',kappasvdRev)

epsperturb=np.random.random(2)*0.05
kappaSVD0=kappasvdRev+epsperturb
resultSVDk= fsolve(FPfuncSVD,kappaSVD0,data)
xFPSVD= m@np.reshape(resultSVDk,(2,1))

print('reconstructed, EI:',np.squeeze(xFPEI),'SVD:',np.squeeze(xFPSVD),'; ')
print('Directly solve X:',resultx)


# calculate the derivatives, based on resultx
derivephi=derivtransfer(resultx,type='tanhp')
diagdphi=np.zeros((2,2))
diagdphi[0,0],diagdphi[1,1]=derivephi[0],derivephi[1]
Jt_conn = (m@n.T)@diagdphi
Jt_ei=(N.T@diagdphi)@M
Jt_svd=(n.T@diagdphi)@m 
compsigtrans=np.zeros((2,3))
_,compsigtrans[:,0],rvecht=la.svd(Jt_conn)
_,compsigtrans[:,1],rvechtei=la.svd(Jt_ei)
_,compsigtrans[:,2],rvechtsvd=la.svd(Jt_svd)
print('comparing singular values of transient matrices',compsigtrans)
compeigtrans=np.zeros((2,3))
compeigtrans[:,0],eigveccon=la.eig(Jt_conn)
compeigtrans[:,1],eigvecei=la.eig(Jt_ei)
compeigtrans[:,2],eigvecsvd=la.eig(Jt_svd)
print('comparing eigen values of transient matrices',compeigtrans)
timeparams=[10,0.001]

# dynx,devx=dynxact(Jei,np.squeeze(resultx),0.1*np.squeeze(rvecht[0,:]),timeparams,type='tanhp')
# dynkei,dynx_kei,devkappaei=dynkappa(M,N,np.squeeze(resultEIk),0.1*np.squeeze(rvechtei[0,:]),timeparams,type='tanhp',express='EI')
# dynksvd,dynx_ksvd,devkappasvd=dynkappa(m,n,np.squeeze(resultSVDk),0.1*np.squeeze(rvechtsvd[0,:]),timeparams,type='tanhp',express='SVD')

print('eigenvector:',eigveccon[:,0])
print('singularvector:',rvecht[:,0])
dynx,devx=dynxact(Jei,np.squeeze(resultx),0.1*np.squeeze(eigveccon[:,0]),timeparams,type='tanhp')
dynkei,dynx_kei,devkappaei=dynkappa(M,N,np.squeeze(resultEIk),0.1*np.squeeze(eigvecei[:,0]),timeparams,type='tanhp',express='EI')
dynksvd,dynx_ksvd,devkappasvd=dynkappa(m,n,np.squeeze(resultSVDk),0.1*np.squeeze(eigvecsvd[:,0]),timeparams,type='tanhp',express='SVD')



reversely kappa_SVD: [0.17442263 2.01746641]
reconstructed, EI: [0.82472918 0.13489005] SVD: [0.82472918 0.13489005] ; 
Directly solve X: [0.82472918 0.13489005]
comparing singular values of transient matrices [[2.45003447 2.45003447 2.64125722]
 [0.2080968  0.2080968  0.19303092]]
comparing eigen values of transient matrices [[ 0.29010286 -1.75746054 -1.75746054]
 [-1.75746054  0.29010286  0.29010286]]
eigenvector: [0.94536565 0.32601194]
singularvector: [-0.40316777 -0.91512609]


In [80]:
# Plot the results
tt=np.arange(0,timeparams[0],timeparams[1])
fig=plt.figure()

ax1=fig.add_subplot(321)
ax1.plot(tt,dynx[0,:],'r',label='E neurons (conn)')
ax1.plot(tt,dynx[1,:],'b',label='I neurons (conn)')
# ax1.plot(tt,np.sqrt(dynx[0,:]**2+dynx[1,:]**2),'k--')
# ax1.set_ylim([-2.5,2.5])
plt.legend()
ax2=fig.add_subplot(323)
# ax2.plot(tt,dynkei[0,:],'r',label='E neurons (EI)')
# ax2.plot(tt,dynkei[1,:],'b',label='I neurons (EI)')
# ax2.plot(tt,np.sqrt(dynkei[0,:]**2+dynkei[1,:]**2),'k--')
ax2.plot(tt,dynx_kei[0,:],'r',label='E neurons (EI)')
ax2.plot(tt,dynx_kei[1,:],'b',label='I neurons (EI)')
# ax2.set_ylim([-2.5,2.5])
plt.legend()

ax3=fig.add_subplot(325)
# ax3.plot(tt,dynksvd[0,:],'r',label='E neurons (SVD)')
# ax3.plot(tt,dynksvd[1,:],'b',label='I neurons (SVD)')
# ax3.plot(tt,np.sqrt(dynksvd[0,:]**2+dynksvd[1,:]**2),'k--')
ax3.plot(tt,dynx_ksvd[0,:],'r',label='E neurons (SVD)')
ax3.plot(tt,dynx_ksvd[1,:],'b',label='I neurons (SVD)')
# ax3.set_ylim([-1.5,3.5])
plt.legend()

ax4=fig.add_subplot(322)
ax4.plot(dynx[0,:],dynx[1,:],'r',label='X trajectory')
ax4.plot(resultx[0],resultx[1],'k*')
ax4.plot([resultx[0],resultx[0]+0.1*eigveccon[0,0]],[resultx[1],resultx[1]+0.1*eigveccon[1,0]],'r--')
ax4.plot([resultx[0],resultx[0]+0.1*eigveccon[0,1]],[resultx[1],resultx[1]+0.1*eigveccon[1,1]],'b--')
# ax1.set_ylim([-2.5,2.5])
plt.legend()
ax5=fig.add_subplot(324)
# ax5.plot(dynkei[0,:],dynkei[1,:],'r',label='kappa trajectory (EI)')
ax5.plot(dynx_kei[0,:],dynx_kei[1,:],'b',label='X trajectory (EI)')
# ax5.plot(results[0],results[1],'k*')
# ax2.plot(tt,dynx_kei[0,:],'r',label='E neurons (EI)')
# ax2.plot(tt,dynx_kei[1,:],'b',label='I neurons (EI)')
# ax2.set_ylim([-2.5,2.5])
plt.legend()

ax6=fig.add_subplot(326)
# ax6.plot(dynksvd[0,:],dynksvd[1,:],'r',label='kappa trajectory (SVD)')
ax6.plot(dynx_ksvd[0,:],dynx_ksvd[1,:],'b',label='X trajectory (SVD)')
# ax6.plot(resultSVD[0],resultSVD[1],'k*')
# ax2.plot(tt,dynx_kei[0,:],'r',label='E neurons (EI)')
# ax2.plot(tt,dynx_kei[1,:],'b',label='I neurons (EI)')
# ax2.set_ylim([-2.5,2.5])
plt.legend()

In [77]:
print(eigveccon)

[[0.99997904 0.60385194]
 [0.00647422 0.7970965 ]]


In [78]:
fig=plt.figure()

ax1=fig.add_subplot(321)
ax1.plot(tt,dynx[0,:],'r',label='E neurons (conn)')
ax1.plot(tt,dynx[1,:],'b',label='I neurons (conn)')
# ax1.plot(tt,np.sqrt(dynx[0,:]**2+dynx[1,:]**2),'k--')
# ax1.set_ylim([-2.5,2.5])
plt.legend()
ax2=fig.add_subplot(323)
ax2.plot(tt,dynkei[0,:],'r',label=r'$\kappa_1$ (EI)')
ax2.plot(tt,dynkei[1,:],'b',label=r'$\kappa_2$ (EI)')
# ax2.set_ylim([-2.5,2.5])
plt.legend()

ax3=fig.add_subplot(325)
ax3.plot(tt,dynksvd[0,:],'r',label=r'$\kappa_1$ (SVD)')
ax3.plot(tt,dynksvd[1,:],'b',label=r'$\kappa_2$ (SVD)')
# ax3.plot(tt,np.sqrt(dynksvd[0,:]**2+dynksvd[1,:]**2),'k--')
# ax3.set_ylim([-1.5,3.5])
plt.legend()

ax4=fig.add_subplot(322)
ax4.plot(dynx[0,:],dynx[1,:],'r',label='X trajectory')
ax4.plot(resultx[0],resultx[1],'k*')
# ax1.set_ylim([-2.5,2.5])
plt.legend()
ax5=fig.add_subplot(324)
ax5.plot(dynkei[0,:],dynkei[1,:],'r',label=r'$\kappa$ trajectory (EI)')
# ax5.plot(dynx_kei[0,:],dynx_kei[1,:],'b',label='X trajectory (EI)')
ax5.plot(resultEIk[0],resultEIk[1],'k*')
# ax2.plot(tt,dynx_kei[0,:],'r',label='E neurons (EI)')
# ax2.plot(tt,dynx_kei[1,:],'b',label='I neurons (EI)')
# ax2.set_ylim([-2.5,2.5])
plt.legend()

ax6=fig.add_subplot(326)
ax6.plot(dynksvd[0,:],dynksvd[1,:],'r',label=r'$\kappa$ trajectory (SVD)')
# ax6.plot(dynx_ksvd[0,:],dynx_ksvd[1,:],'b',label='X trajectory (SVD)')
ax6.plot(resultSVDk[0],resultSVDk[1],'k*')
# ax2.plot(tt,dynx_kei[0,:],'r',label='E neurons (EI)')
# ax2.plot(tt,dynx_kei[1,:],'b',label='I neurons (EI)')
# ax2.set_ylim([-2.5,2.5])
plt.legend()

# Fixed Points
Finding the fixed points $\kappa^*$ using different representations:
* basis vectors using SVD
* basis vectors from E-I perspective

Note:
* plot $\kappa_1,\ \kappa_2$ separately
* plot $\sigma_1,\ \sigma_2$ separately, and how they change with parameters $a,\ b$
* if $J_E$ is not large enough, then changing $a,\ b$ can change the number of FPs
* Also check different transfer functions

** $\phi(x)=tanh(x)$ transfer function **

In [15]:
def FPfuncSVD(x,*data):
    JE,JI,a,b,tfunc=data
    if tfunc=='tanhp':
        x0 = float(x[0])
        x1 = float(x[1])
        # calculate SVD
        Jt = np.zeros((2,2))
        Jt[:,0],Jt[:,1]=JE,-JI
        Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
        lvec,sv,rvech=la.svd(Jt)
        m,n=lvec,rvech.T
        m[:,0]*=sv[0]
        m[:,1]*=sv[1]
        if n[0,0]<0:
            n[:,0]*=(-1)   
            m[:,0]*=(-1)
        if n[0,1]<0:
            n[:,1]*=(-1)   
            m[:,1]*=(-1)
        resp0=np.tanh(x0*m[0,0]+x1*m[0,1])+1
        resp1=np.tanh(x0*m[1,0]+x1*m[1,1])+1
        epszero=1e-6
        if sv[1]<epszero:
            return [
                x0-n[0,0]*resp0-n[1,0]*resp1,
                x1
            ]
        else:
            return [
                x0-n[0,0]*resp0-n[1,0]*resp1,
                x1-n[0,1]*resp0-n[1,1]*resp1
            ]
    elif tfunc=='tanh':
        x0 = float(x[0])
        x1 = float(x[1])
        # calculate SVD
        Jt = np.zeros((2,2))
        Jt[:,0],Jt[:,1]=JE,-JI
        Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
        lvec,sv,rvech=la.svd(Jt)
        m,n=lvec,rvech.T
        m[:,0]*=sv[0]
        m[:,1]*=sv[1]
        # print('sigma2:',sv[1])
        if n[0,0]<0:
            n[:,0]*=(-1)   
            m[:,0]*=(-1)
        if n[0,1]<0:
            n[:,1]*=(-1)   
            m[:,1]*=(-1)
        resp0=(np.tanh(x0*m[0,0]+x1*m[0,1])+0.0)
        resp1=(np.tanh(x0*m[1,0]+x1*m[1,1])+0.0)
        epszero=1e-6
        if sv[1]<epszero:
            return [
                x0-n[0,0]*resp0-n[1,0]*resp1,
                x1
            ]
        else:
            return [
                x0-n[0,0]*resp0-n[1,0]*resp1,
                x1-n[0,1]*resp0-n[1,1]*resp1
            ]


In [16]:
# 2 by 2 connectivity matrix, rank-1 structure
# transfer function \phi(x)=tanh(x)
'''
J = np.zeros((2,2))
'''
JE,JI,a,b=1.0,1.6,-0.20,1.20
jeseries = np.linspace(0.2,1.6,num =80)  
nlen=len(jeseries)
M=np.array([[1,1],[1,-1]])
xFPseries = np.zeros((nlen,2,2))
svdvalues = np.zeros((nlen,2))
svdvec = np.zeros((nlen,2,4))
kappaMN,kappamnSVD=np.zeros((nlen,2)),np.zeros((nlen,2))
epsErr=1e-5
for idxje,JE in enumerate(jeseries):
    # calculate SVD
    Jt = np.zeros((2,2))
    Jt[:,0],Jt[:,1]=JE,-JI
    Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
    if (np.min(Jt[:,0])<0.0) or (np.max(Jt[:,1])>0.0):
        xFPseries[idxje,:,:]=np.nan
        kappaMN[idxje,:]=np.nan
        kappamnSVD[idxje,:]=np.nan
        continue   
    lvec,sv,rvech=la.svd(Jt)
    svdvalues[idxje,:]=sv
    m,n=lvec,rvech.T
            
    m[:,0]*=sv[0]
    m[:,1]*=sv[1]
    if n[0,0]<0:
        n[:,0]*=(-1)   
        m[:,0]*=(-1)
    if n[0,1]<0:
        n[:,1]*=(-1)   
        m[:,1]*=(-1)
    svdvec[idxje,:,:2]=m
    svdvec[idxje,:,2:]=n

    N=np.array([[JE,-a],[-JI,b]])
    data=(JE,JI,a,b,'tanh')
    x0=[10.0,-2.0]
    results = fsolve(FPfunc,x0,data)
    kappaMN[idxje,:]=results
    xFP= M@np.reshape(results,(2,1))
    # check the solution
    errE,errI=checkXbase(xFP,Jt,type='tanh')
    if errE>epsErr or errI>epsErr:
        xFPseries[idxje,0,:]=np.nan
        kappaMN[idxje,:]=np.nan
        continue
    xFPseries[idxje,0,:]=xFP[:,0]
    
    resultSVD= fsolve(FPfuncSVD,x0,data)
    kappamnSVD[idxje,:]=resultSVD
    xFPSVD= m@np.reshape(resultSVD,(2,1))
    # check the solution
    errE,errI=checkXbase(xFPSVD,Jt,type='tanh')
    if errE>epsErr or errI>epsErr:
        xFPseries[idxje,1,:]=np.nan
        kappamnSVD[idxje,:]=np.nan
        continue
    xFPseries[idxje,1,:]=xFPSVD[:,0]
    # check eig
    Xderive=derivtransfer(xFPSVD,type='tanhp')
    Jcheck=Jt.copy()
    Jcheck[:,0]*=Xderive[0]
    Jcheck[:,1]*=Xderive[1]
    eigv,eigvec=la.eig(Jcheck)
    if eigv[0]>1.0:
        xFPseries[idxje,:,:]=np.nan
        kappaMN[idxje,:]=np.nan
        kappamnSVD[idxje,:]=np.nan
        continue


In [17]:
fig=plt.figure()

ax1=fig.add_subplot(131)
ax1.plot(jeseries,xFPseries[:,0,0],'r')
ax1.plot(jeseries,xFPseries[:,0,1],'b')
ax1.set_xlabel(r'$J_E$',fontsize=14)
ax1.set_ylabel(r'$x_{E/I}$',fontsize=14)
# ax2=fig.add_subplot(131)
# ax2.plot(jeseries,xFPseries[:,1,0],'r')
# ax2.plot(jeseries,xFPseries[:,1,1],'b')

ax3=fig.add_subplot(132)
ax3.plot(jeseries,kappaMN[:,0],'r')
ax3.plot(jeseries,kappaMN[:,1],'b')
ax3.set_xlabel(r'$J_E$',fontsize=14)
ax3.set_ylabel(r'$\kappa^{EI}$',fontsize=14)
ax3.set_title(r'$J_I=$'+str(JI)+', $a,\ b=$'+str(a)+','+str(b),fontsize=16)

ax4=fig.add_subplot(133)
ax4.plot(jeseries,kappamnSVD[:,0],'r')
ax4.plot(jeseries,kappamnSVD[:,1],'b')
ax4.set_xlabel(r'$J_E$',fontsize=14)
ax4.set_ylabel(r'$\kappa^{SVD}$',fontsize=14)

Text(0, 0.5, '$\\kappa^{SVD}$')

In [18]:
fig=plt.figure()
ax1=fig.add_subplot(131)
ax1.plot(jeseries,xFPseries[:,0,0],'r--')
ax1.plot(jeseries,xFPseries[:,0,1],'b--')

# ax2=fig.add_subplot(131)
# ax2.plot(jeseries,xFPseries[:,1,0],'r')
# ax2.plot(jeseries,xFPseries[:,1,1],'b')

ax3=fig.add_subplot(132)
ax3.plot(jeseries,kappaMN[:,0],'r--')
ax3.plot(jeseries,kappaMN[:,1],'b--')
ax4=fig.add_subplot(133)
ax4.plot(jeseries,kappamnSVD[:,0],'r--')
ax4.plot(jeseries,kappamnSVD[:,1],'b--')

In [19]:
JE,JI,a,b=1.6,1.2,0.0,0.0
aseries = np.linspace(-0.8,0.8,num =17)  
bseries = np.linspace(-0.8,0.8,num =17)  
nlen=len(aseries)
M=np.array([[1,1],[1,-1]])
xFPseries = np.zeros((nlen,nlen,2,2))
svdvalues = np.zeros((nlen,nlen,2))
kappaMN,kappamnSVD=np.zeros((nlen,nlen,2)),np.zeros((nlen,nlen,2))
epsErr=1e-5
for idxa,a in enumerate(aseries):
    for idxb, b in enumerate(bseries):
        # calculate SVD
        Jt = np.zeros((2,2))
        Jt[:,0],Jt[:,1]=JE,-JI
        Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
        if (np.min(Jt[:,0])<0.0) or (np.max(Jt[:,1])>0.0):
            xFPseries[idxa,idxb,:,:]=np.nan
            kappaMN[idxa,idxb,:]=np.nan
            kappamnSVD[idxa,idxb,:]=np.nan
            continue   
        lvec,sv,rvech=la.svd(Jt)
        svdvalues[idxa,idxb,:]=sv
        m,n=lvec,rvech.T
                
        m[:,0]*=sv[0]
        m[:,1]*=sv[1]
        if n[0,0]<0:
            n[:,0]*=(-1)   
            m[:,0]*=(-1)
        if n[0,1]<0:
            n[:,1]*=(-1)   
            m[:,1]*=(-1)

        N=np.array([[JE,-a],[-JI,b]])
        data=(JE,JI,a,b,'tanh')
        x0=[10.0,-2.0]
        results = fsolve(FPfunc,x0,data)
        kappaMN[idxa,idxb,:]=results
        xFP= M@np.reshape(results,(2,1))
        xFPseries[idxa,idxb,0,:]=xFP[:,0]
        
        resultSVD= fsolve(FPfuncSVD,x0,data)
        kappamnSVD[idxa,idxb,:]=resultSVD
        xFPSVD= m@np.reshape(resultSVD,(2,1))
        xFPseries[idxa,idxb,1,:]=xFPSVD[:,0]
        # # check eig
        # Xderive=derivtransfer(xFPSVD,type='tanhp')
        # Jcheck=Jt.copy()
        # Jcheck[:,0]*=Xderive[0]
        # Jcheck[:,1]*=Xderive[1]
        # eigv,eigvec=la.eig(Jcheck)
        # if eigv[0]>1.0:
        #     xFPseries[idxa,idxb,:,:]=np.nan
        #     kappaMN[idxa,idxb,:]=np.nan
        #     kappamnSVD[idxa,idxb,:]=np.nan
        #     continue

In [20]:
xFPseriesP,kappaMNP,kappamnSVDP=xFPseries,kappaMN,kappamnSVD
# second one, finding the negative
xFPseries = np.zeros((nlen,nlen,2,2))
svdvalues = np.zeros((nlen,nlen,2))
kappaMN,kappamnSVD=np.zeros((nlen,nlen,2)),np.zeros((nlen,nlen,2))
epsErr=1e-5
for idxa,a in enumerate(aseries):
    for idxb, b in enumerate(bseries):
        # calculate SVD
        Jt = np.zeros((2,2))
        Jt[:,0],Jt[:,1]=JE,-JI
        Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
        if (np.min(Jt[:,0])<0.0) or (np.max(Jt[:,1])>0.0):
            xFPseries[idxa,idxb,:,:]=np.nan
            kappaMN[idxa,idxb,:]=np.nan
            kappamnSVD[idxa,idxb,:]=np.nan
            continue   
        lvec,sv,rvech=la.svd(Jt)
        svdvalues[idxa,idxb,:]=sv
        m,n=lvec,rvech.T
                
        m[:,0]*=sv[0]
        m[:,1]*=sv[1]
        if n[0,0]<0:
            n[:,0]*=(-1)   
            m[:,0]*=(-1)
        if n[0,1]<0:
            n[:,1]*=(-1)   
            m[:,1]*=(-1)

        N=np.array([[JE,-a],[-JI,b]])
        data=(JE,JI,a,b,'tanh')
        x0=[-10.0,2.0]
        results = fsolve(FPfunc,x0,data)
        kappaMN[idxa,idxb,:]=results
        xFP= M@np.reshape(results,(2,1))
        xFPseries[idxa,idxb,0,:]=xFP[:,0]
        
        resultSVD= fsolve(FPfuncSVD,x0,data)
        kappamnSVD[idxa,idxb,:]=resultSVD
        xFPSVD= m@np.reshape(resultSVD,(2,1))
        xFPseries[idxa,idxb,1,:]=xFPSVD[:,0]
        # # check eig
        # Xderive=derivtransfer(xFPSVD,type='tanhp')
        # Jcheck=Jt.copy()
        # Jcheck[:,0]*=Xderive[0]
        # Jcheck[:,1]*=Xderive[1]
        # eigv,eigvec=la.eig(Jcheck)
        # if eigv[0]>1.0:
        #     xFPseries[idxa,idxb,:,:]=np.nan
        #     kappaMN[idxa,idxb,:]=np.nan
        #     kappamnSVD[idxa,idxb,:]=np.nan
        #     continue
xFPseriesN,kappaMNN,kappamnSVDN=xFPseries,kappaMN,kappamnSVD

In [21]:
# mdict = {'positive_x': xFPseriesP,
#         'kappaMNP': kappaMNP,
#         'kappamnSVDP':kappamnSVDP,
#         'negative_x':xFPseriesN,
#         'kappaMNN':kappaMNN,
#         'kappamnSVDN':kappamnSVDN}
# savemat('FPstan_ORG2FPs.mat', mdict)

# JE,JI=1.6,1.2; JE,JI=2.6,1.2


In [22]:
fig=plt.figure()
ax1=fig.add_subplot(121)
ax1.imshow(xFPseriesP[:,:,0,0],cmap='jet',vmin=-2.5,vmax=2.5)
ax1.set_xlabel('b')
ax1.set_ylabel('a')
ax2=fig.add_subplot(122)
ax2.imshow(xFPseriesN[:,:,0,0],cmap='jet',vmin=-2.5,vmax=2.5)
ax2.set_xlabel('b')
# ax1=fig.add_subplot(211)
# ax1.set_xlabel(bseries)
# ax1.set_ylabel(aseries)


Text(0.5, 0, 'b')

** $\phi(x)=tanh(x)+1$ transfer function **

In [23]:
# transfer function \phi(x)=tanh(x)+1
'''
J = np.zeros((2,2))
'''
JE,JI,a,b=1.6,1.2,0.0,0.0
jeseries = np.linspace(2.3,2.35,num =1)  
nlen=len(jeseries)
M=np.array([[1,1],[1,-1]])
xFPseries = np.zeros((nlen,2,2))
svdvalues = np.zeros((nlen,2))
svdvec = np.zeros((nlen,2,4))
kappaMN,kappamnSVD=np.zeros((nlen,2)),np.zeros((nlen,2))
epsErr=1e-5
for idxje,JE in enumerate(jeseries):
    # calculate SVD
    Jt = np.zeros((2,2))
    Jt[:,0],Jt[:,1]=JE,-JI
    Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
    if (np.min(Jt[:,0])<0.0) or (np.max(Jt[:,1])>0.0):
        xFPseries[idxje,:,:]=np.nan
        kappaMN[idxje,:]=np.nan
        kappamnSVD[idxje,:]=np.nan
        continue   
    lvec,sv,rvech=la.svd(Jt)
    svdvalues[idxje,:]=sv
    m,n=lvec,rvech.T
            
    m[:,0]*=sv[0]
    m[:,1]*=sv[1]
    if n[0,0]<0:
        n[:,0]*=(-1)   
        m[:,0]*=(-1)
    if n[0,1]<0:
        n[:,1]*=(-1)   
        m[:,1]*=(-1)
    svdvec[idxje,:,:2]=m
    svdvec[idxje,:,2:]=n

    N=np.array([[JE,-a],[-JI,b]])
    data=(JE,JI,a,b,'tanhp')
    x0=[-0.030,-.320]
    results = fsolve(FPfunc,x0,data)
    kappaMN[idxje,:]=results
    xFP= M@np.reshape(results,(2,1))
    # check the solution
    errE,errI=checkXbase(xFP,Jt,type='tanhp')
    if errE>epsErr or errI>epsErr:
        xFPseries[idxje,0,:]=np.nan
        kappaMN[idxje,:]=np.nan
        continue
    xFPseries[idxje,0,:]=xFP[:,0]
    
    resultSVD= fsolve(FPfuncSVD,x0,data)
    kappamnSVD[idxje,:]=resultSVD
    xFPSVD= m@np.reshape(resultSVD,(2,1))
    # check the solution
    errE,errI=checkXbase(xFPSVD,Jt,type='tanhp')
    if errE>epsErr or errI>epsErr:
        xFPseries[idxje,1,:]=np.nan
        kappamnSVD[idxje,:]=np.nan
        continue
    xFPseries[idxje,1,:]=xFPSVD[:,0]


In [24]:
# reversely calculate kappaSVD
def RevfuncSVD(x,*data):
    m,xFP,tfunc=data
    if tfunc=='tanhp':
        k0 = float(x[0])
        k1 = float(x[1])
        rate1=tanhpfunc(xFP[0])
        rate2=tanhpfunc(xFP[1])
    
        return [
            k0*m[0,0]+k1*m[0,1]-xFP[0],
            k0*m[1,0]+k1*m[1,1]-xFP[1]
        ]

In [25]:
# Noncline analysis --- intersection of two lines
JE,JI,a,b=1.6,1.2,-0.575,-0.65
Jt = np.zeros((2,2))
Jt[:,0],Jt[:,1]=JE,-JI
Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
# Use neuronal responses X
xE,xI=np.linspace(-5.0,5.0,320),np.linspace(-5.0,5.0,320)
[XE,XI]=np.meshgrid(xE,xI)
# Surface of rhs in ODEs
rhsE,rhsI=Jt[0,0]*tanhpfunc(XE)+Jt[0,1]*tanhpfunc(XI),Jt[1,0]*tanhpfunc(XE)+Jt[1,1]*tanhpfunc(XI)
# rhsE,rhsI=Jt[0,0]*np.tanh(XE)+Jt[0,1]*np.tanh(XI),Jt[1,0]*np.tanh(XE)+Jt[1,1]*np.tanh(XI)
epsdis = 1*1e-4#((xE[1]-xE[0])/5.0)**2
# finding the intersection
distrhsE=(rhsE-XE)**2
intersectE=np.where(distrhsE<epsdis)
distrhsI=(rhsI-XI)**2
intersectI=np.where(distrhsI<epsdis)

## plot intersection line
fig=plt.figure()
ax1=plt.axes()
ax1.scatter(xE[intersectI[1]],xI[intersectI[0]],c='blue',s=1,label=r'Nullcline $x_I$')
ax1.scatter(xE[intersectE[1]],xI[intersectE[0]],c='red',s=1,label=r'Nullcline $x_E$')
ax1.set_xlabel(r'$x_E$')
ax1.set_ylabel(r'$x_I$')
ax1.set_title(r'$J_E=$'+str(JE)+r',$J_I=$'+str(JI)+r',$a=$'+str(a)+r',$b=$'+str(b))
plt.axis('square')
plt.legend()
plt.grid()

# plt.figure()
# c=plt.contour(XE,XI,rhsE,20,cmap='rainbow')
# plt.clabel(c,inline=True,fontsize=10)
# d=plt.contour(XE,XI,XE,8,colors='BLACK')
# plt.clabel(d,inline=True,fontsize=10)

# plt.figure()
# c1=plt.contour(XE,XI,rhsI,8,cmap='rainbow')
# plt.clabel(c1,inline=True,fontsize=10)
# d1=plt.contour(XE,XI,XI,8,colors='BLUE')
# plt.clabel(d1,inline=True,fontsize=10)

In [26]:
lvec,sv,rvech=la.svd(Jt)
m,n=lvec,rvech.T
        
m[:,0]*=sv[0]
m[:,1]*=sv[1]
if n[0,0]<0:
    n[:,0]*=(-1)   
    m[:,0]*=(-1)
if n[0,1]<0:
    n[:,1]*=(-1)   
    m[:,1]*=(-1)
M=np.array([[1,1],[1,-1]])
N=np.array([[JE,-a],[-JI,b]])
data=(JE,JI,a,b,'tanhp')


x0=[0.428,0.0120]
kappaEI0=[np.sum(x0)/2.0,(x0[0]-x0[1])/2.0]
resultEIk = fsolve(FPfunc,kappaEI0,data)
xFPEI= M@np.reshape(resultEIk,(2,1))

resultx = fsolve(FPXfunc,x0,data)

# reversely find the kappa SVD
xrev0=[0,0]
datarev=(m,resultx,'tanhp')
kappasvdRev=fsolve(RevfuncSVD,xrev0,datarev)
print('reversely kappa_SVD:',kappasvdRev)

epsperturb=np.random.random(2)*0.05
kappaSVD0=kappasvdRev+epsperturb
resultSVDk= fsolve(FPfuncSVD,kappaSVD0,data)
xFPSVD= m@np.reshape(resultSVDk,(2,1))

print('reconstructed, EI:',np.squeeze(xFPEI),'SVD:',np.squeeze(xFPSVD),'; ')
print('Directly solve X:',resultx)


reversely kappa_SVD: [-0.30798046  0.80313033]
reconstructed, EI: [-0.94702967 -0.18257896] SVD: [-0.94702967 -0.18257896] ; 
Directly solve X: [-0.94702967 -0.18257896]


In [27]:
# test the dynamics
xp=-0.001*np.ones_like(resultx)#np.random.random((2))#
timeparams=[80,0.001]
dynx,devx=dynxact(Jt,resultx,xp,timeparams,type='tanhp')

In [28]:
tt=np.arange(0,timeparams[0],timeparams[1])
fig=plt.figure(3)
comparefp=np.ones_like(dynx)
comparefp[0,:]*=resultx[0]
comparefp[1,:]*=resultx[1]
ax1=fig.add_subplot(221)
ax1.plot(tt,dynx[0,:],'r',label='E neurons (line attractor)')
# ax1.plot(tt,dynx[1,:],'b',label='I neurons (conn)')
ax1.plot(tt,comparefp[0,:],'r--',label='FP')
ax1.set_xlabel(r'$J_E=$'+str(JE)+r',$J_I=$'+str(JI)+r',$a=$'+str(a)+r',$b=$'+str(b))
# plt.legend()
# ax1.plot(tt,comparefp[1,:],'b--')
ax2=fig.add_subplot(222)
ax2.plot(tt,dynx[1,:],'r',label='I neurons (line attractor)')
# ax1.plot(tt,dynx[1,:],'b',label='I neurons (conn)')
ax2.plot(tt,comparefp[1,:],'r--',label='FP')
# plt.legend()

In [29]:
tt=np.arange(0,timeparams[0],timeparams[1])
fig=plt.figure(3)
comparefp=np.ones_like(dynx)
comparefp[0,:]*=resultx[0]
comparefp[1,:]*=resultx[1]
ax1=fig.add_subplot(223)
ax1.plot(tt,dynx[0,:],'r',label='E neurons (fixed point)')
# ax1.plot(tt,dynx[1,:],'b',label='I neurons (fixed point)')
ax1.plot(tt,comparefp[0,:],'r--',label='FP')
ax1.set_xlabel(r'$J_E=$'+str(JE)+r',$J_I=$'+str(JI)+r',$a=$'+str(a)+r',$b=$'+str(b))
# plt.legend()

# ax1.plot(tt,comparefp[1,:],'b--')
ax2=fig.add_subplot(224)
ax2.plot(tt,dynx[1,:],'r',label='I neurons (fixed point)')
# ax1.plot(tt,dynx[1,:],'b',label='I neurons (fixed point)')
ax2.plot(tt,comparefp[1,:],'r--',label='FP')
# plt.legend()